# Filter to observed Genotypes, kegg genes

> 

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px

from tqdm import tqdm

from dataG2F.qol import ensure_dir_path_exists, put_cached_result
import dataKEGG.core

In [ ]:
cache_path = '../nbs_artifacts/07_filter_genotypes/'
ensure_dir_path_exists(dir_path = cache_path)

## Load phenotypic data to match

In [ ]:
load_from = '../nbs_artifacts/05_prep_matrices/'
phno = pd.read_csv(load_from+'phno_geno.csv')
phno

,Env,Year,Hybrid,Replicate,Block,Plot,Phno_Idx,Env_Idx,Geno_Idx,Yield_Mg_ha,Stand_Count_plants,Pollen_DAP_days,Silk_DAP_days,Plant_Height_cm,Ear_Height_cm,Root_Lodging_plants,Stalk_Lodging_plants,Grain_Moisture,Twt_kg_m3
0,DEH1_2014,2014,M0088/LH185,1.0,1.0,1.0,0,0,0,5.721725,56.0,63.0,67.0,213.0,79.0,0.0,0.0,20.8,706.664693
1,DEH1_2014,2014,M0143/LH185,1.0,1.0,2.0,1,0,1,11.338246,54.0,61.0,63.0,286.0,172.0,0.0,0.0,25.8,693.792841
2,DEH1_2014,2014,M0003/LH185,1.0,1.0,3.0,2,0,2,6.540810,60.0,63.0,65.0,239.0,92.0,0.0,4.0,20.8,698.941582
3,DEH1_2014,2014,M0035/LH185,1.0,1.0,4.0,3,0,3,10.366857,59.0,61.0,63.0,242.0,118.0,0.0,0.0,23.7,711.813434
4,DEH1_2014,2014,M0052/LH185,1.0,1.0,5.0,4,0,4,10.908814,58.0,63.0,65.0,211.0,92.0,0.0,0.0,19.4,743.993065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133052,WIH3_2022,2022,W10010_0337/LH244,-999.0,-999.0,-999.0,133052,235,4871,11.975018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133053,WIH3_2022,2022,W10010_0346/LH244,-999.0,-999.0,-999.0,133053,235,4872,12.971193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133054,WIH3_2022,2022,W10010_0358/LH244,-999.0,-999.0,-999.0,133054,235,4873,13.499769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133055,WIH3_2022,2022,W10010_0381/LH244,-999.0,-999.0,-999.0,133055,235,4875,10.831640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filtering SNPs to those in a gene (intron or exon)

==NOTE== In this implementation only the coding strand is used. This is fine for using the raw snps but if the gene sequence is to be used then I'll want to flip the %ACGT to have the complement and be able to more directly interpret any changes. 

In [ ]:
geno_path = '../data/zma/g2fc/genotypes_filtered/'
# Useful for converting between the physical location and site
geno_site = pd.read_table(geno_path+'5_Genotype_Data_All_Years_Filter_SiteSummary.txt'
                         ).rename(columns = {'Physical Position':'Position'})
geno_site.head()

,Site Number,Site Name,Chromosome,Position,Number of Taxa,Ref,Alt,Major Allele,Major Allele Gametes,Major Allele Proportion,Major Allele Frequency,Minor Allele,Minor Allele Gametes,Minor Allele Proportion,Minor Allele Frequency,Allele 3,Allele 3 Gametes,Allele 3 Proportion,Allele 3 Frequency,Allele 4,Allele 4 Gametes,Allele 4 Proportion,Allele 4 Frequency,Allele 5,Allele 5 Gametes,Allele 5 Proportion,Allele 5 Frequency,Allele 6,Allele 6 Gametes,Allele 6 Proportion,Allele 6 Frequency,Allele 7,Allele 7 Gametes,Allele 7 Proportion,Allele 7 Frequency,Gametes Missing,Proportion Missing,Number Heterozygous,Proportion Heterozygous,Inbreeding Coefficient,Inbreeding Coefficient Scaled by Missing
0,0,S1_162464,1,162464,4928,C,T,T,6160,0.62500,0.62934,C,3628,0.36810,0.37066,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,68,0.00690,2462,0.50306,TBD,TBD
1,1,S1_565588,1,565588,4928,A,G,G,5857,0.59426,0.60544,A,3817,0.38728,0.39456,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,182,0.01847,3043,0.62911,TBD,TBD
2,2,S1_660286,1,660286,4928,T,C,T,7152,0.72565,0.72801,C,2672,0.27110,0.27199,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,32,0.00325,2286,0.46539,TBD,TBD
3,3,S1_660627,1,660627,4928,T,C,T,7190,0.72950,0.73188,C,2634,0.26725,0.26812,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,32,0.00325,2268,0.46173,TBD,TBD
4,4,S1_666209,1,666209,4928,A,C,A,7190,0.72950,0.73188,C,2634,0.26725,0.26812,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,32,0.00325,2268,0.46173,TBD,TBD


In [ ]:
geno_site['kegg_index'] = np.nan

In [ ]:
kegg_gene_entries = dataKEGG.core.get_gene_entries(species='zma', source='kegg')

In [ ]:
# takes in a position string, returns a list with [chromosome, start, stop, complement]
def parse_kegg_position(POSITION = '7:2815028..2815922'):
    import re
    is_complement = False
    if re.match('.+complement.+', POSITION):
        is_complement = True
        POSITION = POSITION.replace('complement(', '').replace(')', '')

    POSITION = POSITION.replace(':', '..').split('..')

    POSITION = POSITION+['complement' if is_complement else 'coding']
    return(POSITION)

In [ ]:
len(kegg_gene_entries)

37712

In [ ]:
# not all entries have positions and some are unknown
kegg_gene_entries = [e for e in kegg_gene_entries if 'POSITION' in e.keys()]
len(kegg_gene_entries)

37710

In [ ]:
kegg_gene_entries = [e for e in kegg_gene_entries if e['POSITION']!='Unknown']
len(kegg_gene_entries)

35804

In [ ]:
kegg_gene_entry_positions = [parse_kegg_position(e['POSITION']) for e in kegg_gene_entries]

In [ ]:
# drop Plastid genes
is_Pltd = [True if e[0]=='Pltd' else False for e in kegg_gene_entry_positions]


print([kegg_gene_entry_positions[i] for i in range(len(kegg_gene_entry_positions)) if is_Pltd[i] == True  ][0:3])


kegg_gene_entries         = [kegg_gene_entries[i] for i in range(len(kegg_gene_entry_positions)
                                                                ) if is_Pltd[i] != True  ]
kegg_gene_entry_positions = [kegg_gene_entry_positions[i] for i in range(len(kegg_gene_entry_positions)
                                                                        ) if is_Pltd[i] != True  ]

len(kegg_gene_entries)

[['Pltd', '96953', '97973', 'coding'], ['Pltd', '38663', '38974', 'complement'], ['Pltd', '54020', '54092', 'coding']]


35647

In [ ]:
# all position entries must have 4 values
is_right_len = [len(e) == 4 for e in kegg_gene_entry_positions]

kegg_gene_entries         = [kegg_gene_entries[i] for i in range(len(kegg_gene_entry_positions)
                                                                ) if is_right_len[i] ]
kegg_gene_entry_positions = [kegg_gene_entry_positions[i] for i in range(len(kegg_gene_entry_positions)
                                                                        ) if is_right_len[i] ]
len(kegg_gene_entries)

35639

In [ ]:
for i in tqdm(range(len(kegg_gene_entries))):
    ith_chr, ith_loc_start, ith_loc_stop, ith_strand = kegg_gene_entry_positions[i]
    mask = ((geno_site.Chromosome == int(ith_chr)
                ) & (geno_site.Position >= int(ith_loc_start)
                ) & (geno_site.Position <= int(ith_loc_stop)) )

    geno_site.loc[mask, 'kegg_index'] = i

100%|██████████| 35639/35639 [00:24<00:00, 1478.41it/s]


In [ ]:
geno_site.loc[np.isnan(geno_site.kegg_index), 'kegg_index'] = -1
geno_site.kegg_index = geno_site.kegg_index.astype(int)
geno_site

,Site Number,Site Name,Chromosome,Position,Number of Taxa,Ref,Alt,Major Allele,Major Allele Gametes,Major Allele Proportion,Major Allele Frequency,Minor Allele,Minor Allele Gametes,Minor Allele Proportion,Minor Allele Frequency,Allele 3,Allele 3 Gametes,Allele 3 Proportion,Allele 3 Frequency,Allele 4,Allele 4 Gametes,Allele 4 Proportion,Allele 4 Frequency,Allele 5,Allele 5 Gametes,Allele 5 Proportion,Allele 5 Frequency,Allele 6,Allele 6 Gametes,Allele 6 Proportion,Allele 6 Frequency,Allele 7,Allele 7 Gametes,Allele 7 Proportion,Allele 7 Frequency,Gametes Missing,Proportion Missing,Number Heterozygous,Proportion Heterozygous,Inbreeding Coefficient,Inbreeding Coefficient Scaled by Missing,kegg_index
0,0,S1_162464,1,162464,4928,C,T,T,6160,0.62500,0.62934,C,3628,0.36810,0.37066,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,68,0.00690,2462,0.50306,TBD,TBD,-1
1,1,S1_565588,1,565588,4928,A,G,G,5857,0.59426,0.60544,A,3817,0.38728,0.39456,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,182,0.01847,3043,0.62911,TBD,TBD,-1
2,2,S1_660286,1,660286,4928,T,C,T,7152,0.72565,0.72801,C,2672,0.27110,0.27199,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,32,0.00325,2286,0.46539,TBD,TBD,-1
3,3,S1_660627,1,660627,4928,T,C,T,7190,0.72950,0.73188,C,2634,0.26725,0.26812,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,32,0.00325,2268,0.46173,TBD,TBD,-1
4,4,S1_666209,1,666209,4928,A,C,A,7190,0.72950,0.73188,C,2634,0.26725,0.26812,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,32,0.00325,2268,0.46173,TBD,TBD,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125886,125886,S10_151644296,10,151644296,4928,A,G,A,6285,0.63768,0.64277,G,3493,0.35440,0.35723,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,78,0.00791,2387,0.48824,TBD,TBD,14087
125887,125887,S10_151647495,10,151647495,4928,T,C,T,6283,0.63748,0.64283,C,3491,0.35420,0.35717,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,82,0.00832,2385,0.48803,TBD,TBD,14087
125888,125888,S10_151650617,10,151650617,4928,T,C,T,6283,0.63748,0.64283,C,3491,0.35420,0.35717,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,82,0.00832,2385,0.48803,TBD,TBD,14087
125889,125889,S10_151651860,10,151651860,4928,A,G,A,6276,0.63677,0.64264,G,3490,0.35410,0.35736,NaN,0,0.0,0.0,NaN,0,0.0,0.0,NaN,0,0,0,NaN,0,0,0,NaN,0,0,0,90,0.00913,2384,0.48822,TBD,TBD,14087


### Make DataFrames with available entries in KEGG

In [ ]:
kegg_gene_entry_positions_df = pd.DataFrame(
    kegg_gene_entry_positions, 
             columns = ['chr', 'loc_start', 'loc_stop', 'strand']
            ).reset_index().rename(columns = {'index':'kegg_index'})

kegg_gene_entry_positions_df

,kegg_index,chr,loc_start,loc_stop,strand
0,0,7,143196629,143198553,complement
1,1,7,2815028,2815922,coding
2,2,2,231458921,231459472,complement
3,3,2,237292333,237294334,complement
4,4,5,14116779,14123806,coding
...,...,...,...,...,...
35634,35634,7,169630751,169634426,complement
35635,35635,9,148912378,148913059,coding
35636,35636,4,215197632,215198215,coding
35637,35637,1,193920568,193923402,coding


In [ ]:
geno_site_in_genes = geno_site.loc[:, [
    'Site Number',
    'Site Name',
    'Chromosome',
    'Position',
    'kegg_index']].loc[geno_site.kegg_index != -1] # Filter to only those with a kegg index
geno_site_in_genes

,Site Number,Site Name,Chromosome,Position,kegg_index
6,6,S1_681760,1,681760,104
7,7,S1_684017,1,684017,104
8,8,S1_684410,1,684410,104
9,9,S1_686498,1,686498,104
10,10,S1_687094,1,687094,104
...,...,...,...,...,...
125886,125886,S10_151644296,10,151644296,14087
125887,125887,S10_151647495,10,151647495,14087
125888,125888,S10_151650617,10,151650617,14087
125889,125889,S10_151651860,10,151651860,14087


In [ ]:
geno_site_in_genes = geno_site_in_genes.merge(kegg_gene_entry_positions_df)
geno_site_in_genes

,Site Number,Site Name,Chromosome,Position,kegg_index,chr,loc_start,loc_stop,strand
0,6,S1_681760,1,681760,104,1,680758,707960,complement
1,7,S1_684017,1,684017,104,1,680758,707960,complement
2,8,S1_684410,1,684410,104,1,680758,707960,complement
3,9,S1_686498,1,686498,104,1,680758,707960,complement
4,10,S1_687094,1,687094,104,1,680758,707960,complement
...,...,...,...,...,...,...,...,...,...
75414,125886,S10_151644296,10,151644296,14087,10,151633208,151657426,coding
75415,125887,S10_151647495,10,151647495,14087,10,151633208,151657426,coding
75416,125888,S10_151650617,10,151650617,14087,10,151633208,151657426,coding
75417,125889,S10_151651860,10,151651860,14087,10,151633208,151657426,coding


### Use above to filter genotype

In [ ]:
load_from = '../nbs_artifacts/05_prep_matrices/'
ACGT = np.load(load_from+'ACGT.npy')

In [ ]:
ACGT.shape

(4926, 4, 125891)

In [ ]:
geno_site_in_genes

,Site Number,Site Name,Chromosome,Position,kegg_index,chr,loc_start,loc_stop,strand
0,6,S1_681760,1,681760,104,1,680758,707960,complement
1,7,S1_684017,1,684017,104,1,680758,707960,complement
2,8,S1_684410,1,684410,104,1,680758,707960,complement
3,9,S1_686498,1,686498,104,1,680758,707960,complement
4,10,S1_687094,1,687094,104,1,680758,707960,complement
...,...,...,...,...,...,...,...,...,...
75414,125886,S10_151644296,10,151644296,14087,10,151633208,151657426,coding
75415,125887,S10_151647495,10,151647495,14087,10,151633208,151657426,coding
75416,125888,S10_151650617,10,151650617,14087,10,151633208,151657426,coding
75417,125889,S10_151651860,10,151651860,14087,10,151633208,151657426,coding


In [ ]:
# create a lookup table
ACGT_gene_slice_list_to_kegg_lookup = pd.DataFrame(zip(
    [i for i in range(len(geno_site_in_genes.kegg_index.drop_duplicates()))],
    geno_site_in_genes.kegg_index.drop_duplicates()), 
            columns = ['ACGT_gene_slice_list', 'kegg_index'])

ACGT_gene_slice_list_to_kegg_lookup

,ACGT_gene_slice_list,kegg_index
0,0,104
1,1,19664
2,2,14471
3,3,1431
4,4,3721
...,...,...
13934,13934,15080
13935,13935,16813
13936,13936,32084
13937,13937,14087


In [ ]:
geno_site_in_genes.kegg_index.drop_duplicates()

0          104
11       19664
13       14471
14        1431
17        3721
         ...  
75396    15080
75406    16813
75408    32084
75411    14087
75418    29651
Name: kegg_index, Length: 13939, dtype: int64

In [ ]:
filtered_kegg_gene_entries = []

for i in ACGT_gene_slice_list_to_kegg_lookup.kegg_index:
    filtered_kegg_gene_entries += [kegg_gene_entries[i]]

# filtered_kegg_gene_entries[0]

In [ ]:
ACGT_gene_slice_list = []
ACGT_gene_site_name_list = []

for kegg_index in tqdm(geno_site_in_genes.kegg_index.drop_duplicates()):
    # kegg_index = geno_site_in_genes.kegg_index.drop_duplicates()[0]
    site_list = list(geno_site_in_genes.loc[geno_site_in_genes.kegg_index == kegg_index, 'Site Number'])
    site_name_list = list(geno_site_in_genes.loc[geno_site_in_genes.kegg_index == kegg_index, 'Site Name'])
    ACGT_gene_site_name_list += [site_name_list]
    ACGT_gene_slice_list += [ACGT[:, :, site_list]]

100%|██████████| 13939/13939 [00:12<00:00, 1122.37it/s]


In [ ]:
# to be able to work with this I need

# `ACGT_gene_slice_list` ----
# Has selected SNPs
#                      ~~Match this with the kegg index using `ACGT_gene_slice_list_to_kegg_lookup`~~
#                      Now matches `filtered_kegg_gene_entries`
#                      |
# ACGT_gene_slice_list[0] (4926, 4, 11)
#                          |
#                          Match this with non-deduplicated genotype (i.e. get the right match for y)
#                          Use `obs_geno_lookup` in '../nbs_artifacts/01.03_g2fc_prep_matrices/'

# `ACGT_gene_slice_list_to_kegg_lookup` 
# Go from position in list to kegg


# `kegg_gene_entries` (The filtered version) ----
# way to build up the network structure from the relevant 

In [ ]:
[len(e) for e in [ACGT_gene_slice_list, filtered_kegg_gene_entries ]]

In [ ]:
put_cached_result(cache_path+'ACGT_gene_slice_list.pkl', ACGT_gene_slice_list)
put_cached_result(cache_path+'ACGT_gene_site_name_list.pkl', ACGT_gene_site_name_list) # Allows for easier matching with other methods

In [ ]:
put_cached_result(cache_path+'filtered_kegg_gene_entries.pkl', filtered_kegg_gene_entries)

## Visualize available SNPs with respect to KEGG genes

In [ ]:
temp = geno_site.groupby('kegg_index'
                 ).count(
                 ).reset_index(
                 ).loc[:, ['kegg_index', 'Chromosome']
                 ].rename(columns = {'Chromosome':'Count'})
temp

In [ ]:
print('Kegg entries with SNPs: '+str(temp[((temp.kegg_index != -1
                                       ) & (temp.Count > 1))].shape[0]))

In [ ]:
px.histogram(temp.loc[(temp.kegg_index != -1),], 
             x = 'Count',
            title= 'Observed SNPS per ')

In [ ]:
print('Kegg entries with SNPs: '+str(temp[(temp.kegg_index != -1)].shape[0]))

In [ ]:
temp = pd.DataFrame(
    zip(
        range(20),
        [temp[((temp.kegg_index != -1) & (temp.Count > i))].shape[0] for i in range(20)]
    ), columns=['MoreThanXSNPs', 'Genes'])

temp.head()

In [ ]:
# decrease appears to be log linear
px.scatter(temp, x = 'MoreThanXSNPs', y = 'Genes', log_y= True)

### Consider one KEGG index

In [ ]:
kegg_index = 35628

In [ ]:
px.scatter(geno_site.loc[(geno_site.kegg_index == kegg_index)], x = 'Position', y = 'Ref' )